## Création d'un classificateur pour déterminer le type de pneu utilisé pendant un relais
En Formule 1, cinq types de pneus peuvent être utilisés pendant une course : 
<ul>
    <li> Soft : ce pneu, extrêmement rapide, possède la durée de vie la plus courte.</li>
    <li> Medium : ce pneu est moyennement rapide et possède une durée de vie moyenne.</li>
    <li> Hard : ce pneu est le plus lent mais le plus durable.</li>
    <li> Inter : ce pneu est conçu pour être utilisé sur circuit humide.</li>
    <li> Wet : ce pneu est utilisé dans le cas d'un circuit trempé, c'est le plus lent de tous.</li>
</ul>
Nous n'avons pas de labels à notre disposition pour entraîner un modèle de manière supervisée. Il s'agit d'un problème <b>d'apprentissage non supervisé</b>.

Il est raisonnable de penser que réussir à classifier les pneus dans ce scénario est hors de portée pour des étudiants de L3 : nous allons donc utiliser un ensemble d'apprentissage très restreint, ne contenant que les données de pneus pour une saison, recueillies à la main.

![tyre.jpg](attachment:tyre.jpg)

<a href="https://f1metrics.wordpress.com/2014/06/04/who-was-the-best-wet-weather-driver/#bottom">Ce lien recense toutes les courses de Formule 1 qui se sont déroulées sous la pluie</a><br>
Il est possible d'utiliser ces données pour classifier : en effet, si une course ne s'est pas déroulée par temps pluvieux, alors on peut disqualifier d'office les pneux <i>Inter</i> et <i>Wet</i>.

### Intuitions pour développer le modèle

L'objectif est de trouver, pendant une course et pour un pilote donné, quels composés de pneus il a utilisé.
La différence entre deux composés de pneus peut être observée sur deux plans : la durée du relais (longévité du pneu) et le temps au tour (adhérence du pneu).

Il convient de prendre en compte les considérations suivantes : 

- Un pilote Williams ayant chaussé des <i>softs</i> peut avoir, au même moment de la course, un temps au tour supérieur à un pilote Ferrari équipé de <i>hards</i> en raison de la différence de puissance entre les deux monoplaces. Pour modéliser ce phénomène, on peut réutiliser les coefficients de puissance et de forme introduits dans la partie régression linéaire de ce projet.

- Au fur et à mesure de l'avancement de la course, les voitures s'allègent en raison de la diminution de la masse d'essence dans le réservoir : c'est pourquoi le <i>Meilleur tour</i> est très souvent obtenu en fin de course : on peut donc prendre en compte le nombre de tours, sous forme de pourcentage, pour classifier pneus.

- Naturellement,un temps au tour ne peut être représentatif que pour un circuit donné : c'est pour cela qu'il conviendra de stocker l'identifiant du circuit sous forme de one-hot.

Notre dataset sera de la forme suivante:

| circuitId | constructorPower | constructorForm | race_completion | stint_duration | stint_avg_laptime | compound |


En fait l'histoire est plus complexe : Pirelli, le fabricant de pneus officiel de la Formule 1, propose une gamme de 5 composés de pneus pour la saison. Ils possèdent chacun un identifiant, qui s'étend de C1 (gomme la plus dure) à C5 (gomme la plus molle).

Chaque weekend, trois composés sont sélectionnés parmi les 5 disponibles et sont affublés des noms <i>soft, medium et hard</i>. Ainsi, si le choix de pneus est $(C1,C2,C3)$ pour un weekend $w_1$ et $(C3,C4,C5)$ pour un weekend $w_2$, le pneu _soft_ de $w_1$ est exactement le même composé que le _hard_ de $w_2$ !

C'est pourquoi nous devrons associer, pour chaque course, aux pneus leur composé exact et procéder à une classification en cinq classses et non pas trois comme l'intuition nous le suggèrerait.

In [ ]:
import os
import tempfile
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

#import scikitplot as skplt
import warnings
warnings.filterwarnings("ignore")

sns.set()
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
tyres = pd.read_csv("datasets/tyres.csv")
constructorResults = pd.read_csv("datasets/constructor_results.csv", index_col=0)
constructors = pd.read_csv("datasets/constructors.csv", index_col=0)
constructorStandings = pd.read_csv("datasets/constructor_standings.csv", index_col=0)
drivers = pd.read_csv("datasets/drivers.csv", index_col=0)
lapTimes = pd.read_csv("datasets/lap_times.csv")
pitStops = pd.read_csv("datasets/pit_stops.csv")
qualifying = pd.read_csv("datasets/qualifying.csv", index_col=0)
races = pd.read_csv("datasets/races.csv",index_col=0)
results = pd.read_csv("datasets/results.csv", index_col=0)
status = pd.read_csv("datasets/status.csv", index_col=0)
races = races.reindex(np.random.permutation(races.index))

In [ ]:
'''
Ce dictionnaire associe à chaque course de la saison 2019 le choix de pneus effectué par 
Pirelli (données recueillies à la main).


================================================================
UPDATE 16 AVRIL : LE MODELE FONCTIONNE MIEUX AVEC LES 5 COMPOSES 
================================================================


On remarque que la classification en prenant en compte les 5 composés marche moins bien qu'en gardant 
un modèle simple à trois composés.
Il faut donc soit améliorer le modèle soit repasser à 3 composés (soft, medium, hard)
'''
tyre_compound_selection = {1010:[4,3,2], #Australia
                           1011:[3,2,1], #Bahrain
                           1012:[4,3,2], #China
                           1013:[4,3,2], #Azerbaijan
                           1014:[3,2,1], #Spain
                           1015:[5,4,3], #Monaco
                           1016:[5,4,3], #Canada
                           1017:[4,3,2], #France
                           1018:[4,3,2], #Austria
                           1019:[3,2,1], #Great Britain
                           1020:[4,3,2], #Germany (cette course s'est déroulée sous la pluie)
                           1021:[4,3,2], #Hungary
                           1022:[3,2,1], #Belgium
                           1023:[4,3,2], #Italia
                           1024:[5,4,3], #Singapore
                           1025:[4,3,2], #Russia
                           1026:[3,2,1], #Japan
                           1027:[4,3,2], #Mexico
                           1028:[4,3,2], #USA
                           1029:[3,2,1], #Brazil
                           1030:[5,4,3], #Abu Dhabi
                          }

In [ ]:
def getInformation(raceId, driverId):
    '''
    Cette fonction permet d'exécuter quelques lignes de code fastidieuses.
    In : raceId (int)
         driverId (int)
    Out : 6 DataFrames correspondant à l'union de chaque DataFrame importé avec le numéro de course et le numéro de pilote.
    '''
    try:
        race = races.loc[raceId] #Find the race
        driver = drivers.loc[driverId] #Find the driver
        result = results[(results['raceId'] == raceId) & (results['driverId'] == driverId)].iloc[0]
        pits = pitStops[(pitStops['raceId'] == raceId) & (pitStops['driverId'] == driverId)]
        quali = qualifying[(qualifying['raceId'] == raceId) & (qualifying['driverId'] == driverId)]
        laps = lapTimes[(lapTimes['raceId'] == raceId) & (lapTimes['driverId'] == driverId)]
        return(race,driver,result,pits,quali,laps)
    except Exception: #Si on a entré un mauvais couple (raceId, driverId)
        print('Impossible de trouver la course n°{}'.format(raceId))
        return getInformation(raceId-1,driverId)


def getQualifyingPosition(raceId, driverId):
    '''
    In : raceId (int)
         driverId (int)
    Out : le résultat de qualifications du pilote (pas forcément égal à la position sur la grille, il peut 
          y avoir des pénalités) (int)    
    '''
    quali = getInformation(raceId,driverId)[4]
    if not quali.empty:
        return quali['position'].iloc[0]
    else :
        return 'Inconnu'
    
def getFastestLapTime(raceId):
    '''
    In : raceId (int)
    Out : le temps en millisecondes du tour de course le plus rapide (int)
    '''
    result = results[results['raceId'] == raceId]
    mylap = result['fastestLapTime'].min()
    minutes = int(mylap.split(":")[0])
    seconds = int(mylap.split(":")[1].split(".")[0])
    milliseconds = int(mylap.split(":")[1].split(".")[1])
    return (60000*minutes+1000*seconds+milliseconds)

def getQualifyingTime(raceId,driverId):
    '''
    In : raceId (int)
         driverId (int)
    Out : le temps en millisecondes du tour de qualifications le plus rapide (int)
    '''
    try:
        quali = qualifying[(qualifying['raceId'] == raceId) & (qualifying['driverId'] == driverId)]
        quali = quali.fillna(value='9:99.999')
        quali = quali.iloc[0]
        mylap = quali[['q1','q2','q3']]
        mylap = mylap.min()
        minutes = int(mylap.split(":")[0])
        seconds = int(mylap.split(":")[1].split(".")[0])
        milliseconds = int(mylap.split(":")[1].split(".")[1])
        return (60000*minutes+1000*seconds+milliseconds)
    except Exception: #Le pilote n'a pas participé aux qualifications (exemple Albon à Shanghai 2019)
        return getQualifyingTime(raceId, getTeammate(raceId,driverId)) #On retourne le temps de qualifications de son coéquipier.

def getComparison(raceId,driver1,driver2):
    '''
    Cette fonction d'affichage imprime un head-to-head entre deux pilotes passés en paramètres.
    On trace la comparaison de la position ainsi que la comparaison des temps au tour.
    In : raceId (int)
         driver1 (int)
         driver2 (int)
    '''
    (race,driver,result,pits,quali,laps) =getInformation(raceId,driver1)
    (race2,driver2,result2,pits2,quali2,laps2) =getInformation(raceId,driver2)
    if not laps.empty and not laps2.empty:
        fig = plt.figure(figsize=(14,5),dpi=300)
        ax2 = fig.add_subplot(121)
        l1, = ax2.plot(laps['lap'].tolist(), laps['milliseconds'].tolist(), 'blue')
        l2, = ax2.plot(laps2['lap'].tolist(), laps2['milliseconds'].tolist(), 'red')
        ax2.set_ylabel('milliseconds')
        l1.set_label(getDriverName(driver1))
        l2.set_label(getDriverName(driver2.name))
        ax2.legend()
        ax2.title.set_text('Comparaison des temps au tour')
        
        ax1 = fig.add_subplot(122)
        ax1.set_ylabel('position')
        plt.gca().invert_yaxis()
#         plt.fill_between(laps['lap'], laps['position'],color="blue", alpha=0.1)
#         plt.fill_between(laps2['lap'], laps2['position'],color="red", alpha=0.1)
        l3, = ax1.plot(laps['lap'].tolist(), laps['position'].tolist(),color='blue')
        l3.set_label(getDriverName(driver1))
        l4, = ax1.plot(laps2['lap'].tolist(), laps2['position'].tolist(),color='red')
        l4.set_label(getDriverName(driver2.name))
        ax1.legend()
        ax1.title.set_text('Comparaison de la position')
        plt.tight_layout()
    else:
        print('Comparaison non disponible')

In [ ]:
def getWCCPoints(year,constructorId):
    '''
    In : year (int)
         constructorId (int)
    Out : le nombre de points au championnat constructeurs en fin de saison (int)
    '''
    try:
        myRaces = races[races['year'] == year].sort_values('round')
        standings = constructorStandings[constructorStandings['constructorId'] == constructorId]
        standings = standings.merge(myRaces,right_index=True,left_on='raceId')
        return standings['points'].iloc[standings.shape[0]-1]
    except Exception:
        print('Impossible de retrouver les points')

def getWCCResult(year, constructorId):
    '''
    In : year (int)
         constructorId (int)
    Out : le classement au championnat constructeur en fin de saison (int)
    '''
    try:
        myRaces = races[races['year'] == year].sort_values('round')
        standings = constructorStandings.merge(myRaces,right_index=True,left_on='raceId')
        standings = standings[standings['round'] == standings['round'].max()]
        standings = standings[standings['constructorId'] == constructorId]
    #     print("L'équipe "+getConstructorName(constructorId)+'('+str(constructorId)+')'+" a terminé "+str(standings['position'].tolist()[0])+"e")
        return standings['position'].tolist()[0]
    except Exception:
        return 5
    
def getWCCPosition(year, myround, constructorId):
    '''
    In : year (int)
         myround (int) : numéro de la course dans la saison
         constructorId (int)
    Out : la position au championnat constructeurs à un moment donné de la saison (int)
    '''
    try:
        myRaces = races[races['year'] == year].sort_values('round')
        standings = constructorStandings.merge(myRaces,right_index=True,left_on='raceId')
        standings = standings[standings['round'] == myround]
        standings = standings[standings['constructorId'] == constructorId]
        return standings['position'].tolist()[0]
    except Exception:
        return 5 #Si c'est la première course de la saison, on simule toutes les écuries comme égales.
    
    
def getWCCStandings(year):
    '''
    In : year (int)
    Out : le classement constructeurs de cette année (DataFrame)
    '''
    myRaces = races[races['year'] == year].sort_values('round')
    standings = constructorStandings.merge(myRaces,right_index=True,left_on='raceId')
    standings = standings[standings['round'] == standings['round'].max()]
    return standings[['constructorId','position']].sort_values('position',ascending=True)

    

In [ ]:
def getStints(raceId,driverId):
    '''
    Cette fonction donne les différents relais effectués par un pilote pendant une course (tous deux
    passés en paramètre).
    L'intuition nous suggère de prendre, pour chaque relais, la moyenne du temps au tour.
    Plutôt , on prend le tour le plus rapide de ce pilote en qualifications (donc sur le pneu le plus tendre).
    Cela nous permet de retirer l'aspect de différence de puissance entre les voitures puisqu'on compare un pilote 
    avec sa propre performance.
    In : raceId (int)
         driverId (int)
    Out : stintDf (DataFrame) qui comporte les features de chaque relais pour ce pilote.
    '''
    #On crée le DataFrame qu'on va retourner
    stintDf = pd.DataFrame(columns=['circuitId','driverId','constructorId', 'constructorPower','constructorForm', 'race_completion', 'stint_duration','stint_avg_laptime','compound'])
    
    #On récupère les informations de course
    (race,driver,result,pits,quali,laps) = getInformation(raceId, driverId)
    
    #On récupère le composé de pneus de cette course pour le pilote
    tyre = tyres[(tyres['raceId'] == raceId) & (tyres['driverId'] == driverId)]
    
    #On récupère l'année, le constructeur, le circuit, le round, 
    #le tour le plus rapide en qualifications et en course
    year = race['year']
    current_round = race['round']
    qualifyingTimes = getQualifyingTime(raceId, driverId)
    fastestLap = getFastestLapTime(raceId)
    circuit = int(race['circuitId'])
    constructor = int(result['constructorId'])
    
    #On associe à chaque tour son rapport avec le temps de qualifications.
    #Le modèle marche moins bien si on fait cela, donc on ne l'utilise pas pour l'instant.
#     laps['milliseconds'] = laps['milliseconds']/qualifyingTimes
    
    #On crée les coefficients de forme et de puissance.
    power = 1.0/np.power(np.tan(0.1*getWCCResult(year-1,result['constructorId'])),0.7)
    form = 1.0/np.power(np.tan(0.1*getWCCPosition(year,current_round,result['constructorId'])),0.7)
    
    #On retrouve les informations de pit-stop
    race_duration = laps.shape[0]
    pit_laps = pits['lap']
    nb_pits = 0
    laps2 = {0:[]}
    stints = {0:[]}
    for i in range(1,race_duration):
        try:
            if i < pit_laps.iloc[nb_pits]: #Tant qu'un pilote est entre deux pit-stops
                laps2[nb_pits].append(laps['lap'].iloc[i])
                stints[nb_pits].append(laps['milliseconds'].iloc[i])
            elif i == pit_laps.iloc[nb_pits]: #Quand un pilote rentre dans la pitlane
                laps2[nb_pits].append(laps['lap'].iloc[i])
                stints[nb_pits].append(laps['milliseconds'].iloc[i])
                nb_pits += 1
                stints[nb_pits] = []
                laps2[nb_pits] = []
        except IndexError :
            laps2[nb_pits].append(laps['lap'].iloc[i])
            stints[nb_pits].append(laps['milliseconds'].iloc[i])
    for key in stints.keys():
        gap = max(laps2[key])-min(laps2[key])+1 #La durée en tours de ce relais
        stints[key] = np.mean(stints[key]) #La moyenne des rapports des temps au tour
        laps2[key] = np.mean(laps2[key]) #A quel moment de la course ce relais a-t-il eu lieu ?
        compound = tyre[tyre['nbStops'] == key]
        print(compound)
        mycompound = compound['compound'].iloc[0]
        stintDf = stintDf.append({'circuitId':circuit,'driverId':driverId, 'constructorId':constructor,'constructorPower':power,'constructorForm':form,'race_completion':laps2[key],'stint_duration':(1.0*gap/race_duration),'stint_avg_laptime':stints[key],'compound':mycompound}, ignore_index=True)
    return stintDf

In [ ]:
mystints = getStints(1014,844)
display(mystints)

In [ ]:
def prepare_stint_dataset(raceId):
    dataframe = pd.DataFrame(columns=['circuitId', 'constructorPower','constructorForm', 'race_completion', 'stint_duration','stint_avg_laptime','compound'])
    drivers = getDriversInRace(raceId)
    for driver in drivers:
        print('{} : {}\r'.format(driver,getDriverName(driver))),
#         time.sleep(0.25)
        dataframe = dataframe.append(getStints(raceId, driver))
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index', axis=1)
    mycompounds = tyre_compound_selection[raceId]
#     for i in range(0,len(mycompounds)):
#         dataframe['compound'] = np.where(dataframe['compound'] == i+1, mycompounds[i],dataframe['compound'])
    dataframe['compound'] = np.where(dataframe['compound'] == 3,mycompounds[2],dataframe['compound'])
    dataframe['compound'] = np.where(dataframe['compound'] == 2,mycompounds[1],dataframe['compound'])
    dataframe['compound'] = np.where(dataframe['compound'] == 1,mycompounds[0],dataframe['compound'])
    dataframe['driverPerformance'] = dataframe['driverId'].apply(lambda x: computeDriverPerformance(raceId, x))
    return dataframe

In [ ]:
df = prepare_stint_dataset(1014)
# df = df[df['stint_duration'] >= 10]
display(df.head())

In [ ]:
sns.barplot(df['compound'], df['stint_avg_laptime'])
sns.jointplot(df['compound'], df['stint_duration'])

In [ ]:
data = pd.DataFrame(columns=['circuitId', 'constructorPower','constructorForm', 'race_completion', 'stint_duration','stint_avg_laptime','compound'])
for i in range(1010,1019):
    print(i)
    data = data.append(prepare_stint_dataset(i))
data = data.reset_index()
data['stint_avg_laptime'] = (data['stint_avg_laptime'] - data['stint_avg_laptime'].mean()) / data['stint_avg_laptime'].std()
data['stint_duration'] = (data['stint_duration'] - data['stint_duration'].mean()) / data['stint_duration'].std()
data['race_completion'] = (data['race_completion'] - data['race_completion'].mean()) / data['race_completion'].std()

display(data.head())
display(data.describe())

In [ ]:
data.iloc[20]

In [ ]:
# %matplotlib notebook
fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')
data1= data[data['compound'] == 2.0]
data2= data[data['compound'] == 3.0]
data3= data[data['compound'] == 4.0]
data4= data[data['compound'] == 5.0]
# Data for three-dimensional scattered points
zdata1 = data1['race_completion']
xdata1 = data1['stint_avg_laptime']
ydata1 = data1['stint_duration']
ax1.scatter3D(xdata1, ydata1, zdata1, c=(xdata1), cmap='Blues_r');

zdata2 = data2['race_completion']
xdata2 = data2['stint_avg_laptime']
ydata2 = data2['stint_duration']
ax1.scatter3D(xdata2, ydata2, zdata2, c=(xdata2), cmap='autumn');

zdata3 = data3['race_completion']
xdata3 = data3['stint_avg_laptime']
ydata3 = data3['stint_duration']
ax1.scatter3D(xdata3, ydata3, zdata3, c=(xdata3), cmap='Greens');



fig.show()

Comme on peut le voir sur le graphique 3D, il n'y  pas grand chose à prédire. Nous allons tout de même faire de notre mieux.

In [ ]:
#On shuffle le dataframe.
date= data.reindex(np.random.permutation(data.index))
#On crée les ensembles d'entraînement et de test.
train_tyre = date.sample(frac=0.8,random_state=1)
test_tyre = date.drop(train_tyre.index)
train_tyre = train_tyre[['constructorForm', 'circuitId', 'compound', 'constructorPower', 'driverPerformance', 'race_completion', 'stint_avg_laptime', 'stint_duration']]
test_tyre = test_tyre[['constructorForm', 'circuitId', 'compound', 'constructorPower', 'driverPerformance', 'race_completion', 'stint_avg_laptime', 'stint_duration']]

In [ ]:
#Création des labels
train_tyre_label = train_tyre.pop('compound')
test_tyre_label = test_tyre.pop('compound')
#On transforme le circuitId en one-hot.
train_tyre['circuitId'] = pd.Categorical(train_tyre['circuitId'])
train_tyre = pd.get_dummies(train_tyre)
test_tyre['circuitId'] = pd.Categorical(test_tyre['circuitId'])
test_tyre = pd.get_dummies(test_tyre)

In [ ]:
'''
On transforme les DataFrame en tableaux numpy pour pouvoir prédire avec le Random Forest.
'''
tr = train_tyre.to_numpy()
tr_l = train_tyre_label.to_numpy()
te = test_tyre.to_numpy()
te_l = test_tyre_label.to_numpy()

In [ ]:
#On initialise le modèle de RandomForest.

from randomForest import RandomForest
myforest = RandomForest(111,5,3)
myforest.build_forest(tr, tr_l)

In [ ]:
#On crée la prédiction
predicted = myforest.predict(te)
print(predicted)
#On assigne 1 si le modèle a prédit le bon pneu et 0 sinon.
predicted = predicted-te_l
predicted = np.where(predicted != 0, 0, 1)

In [ ]:
print(predicted)
print("La précision est de {}".format((1.*np.sum(predicted))/(1.*predicted.shape[0])))